# Segmenting and clustering neighborhoods in Toronto
## For consistency, We choose North York of Toronto to reproduce the Manhattan neighborhoods from the Lab
### I started from the beginning, wikipedia dataframe to North York neighborhood

In [44]:
import pandas as pd
import matplotlib.cm as cm
import matplotlib.colors as colors
link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df = pd.read_html(link,header=0)[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [7]:
df2=df[~df.Borough.str.contains("Not assigned")]
df2.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [8]:
df2.Neighborhood=df2.Neighborhood.str.replace('/',',')
df2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park , Harbourfront"
5,M6A,North York,"Lawrence Manor , Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"


## Load the geospatial data and merge them with Toronto neighborhood
### We can merge the lat. and long. using simple python dictionary

In [9]:
!wget -q -O 'Geospatial Coordinates.csv' https://cocl.us/Geospatial_data
g1=pd.read_csv('Geospatial Coordinates.csv')
g1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:

d1=dict(zip(g1['Postal Code'],g1.Latitude))
d2=dict(zip(g1['Postal Code'],g1.Longitude))
df2['Latitude']=df2['Postal code'].map(d1)
df2['Longitude']=df2['Postal code'].map(d2)
df2.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal code,Borough,Neighborhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government",43.662301,-79.389494


### Load necessary libraries

In [3]:
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
! conda install -c conda-forge folium geopy --yes
import folium


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    folium-0.10.1              |             py_0          59 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    

### Get the coordinates of Toronto city

In [4]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### Plot the Map of Toronto neighborhoods

In [12]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Create variable with foursquare api credentials

In [13]:
CLIENT_ID = 'ODECZSQYQFUYBVEJ1S2LA3Q3PDMRQSFH1LRMGPIJRMAPTOP4' # your Foursquare ID
CLIENT_SECRET = 'YIM4JORNLBLJHLAWUYRD2DYEMYC4B0CFAJS4Z3RJSZMJRQOM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ODECZSQYQFUYBVEJ1S2LA3Q3PDMRQSFH1LRMGPIJRMAPTOP4
CLIENT_SECRET:YIM4JORNLBLJHLAWUYRD2DYEMYC4B0CFAJS4Z3RJSZMJRQOM


### Get the coordinates of scpecific neighborhood

In [14]:
df2.loc[2, 'Neighborhood']
neighborhood_latitude = df2.loc[2, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df2.loc[2, 'Longitude'] # neighborhood longitude value

neighborhood_name = df2.loc[2, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


### Create url with the coordinate 

In [15]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=ODECZSQYQFUYBVEJ1S2LA3Q3PDMRQSFH1LRMGPIJRMAPTOP4&client_secret=YIM4JORNLBLJHLAWUYRD2DYEMYC4B0CFAJS4Z3RJSZMJRQOM&v=20180604&ll=43.7532586,-79.3296565&radius=500&limit=100'

### Get the info of all neighborhoods around the previous coordinate

In [16]:
import requests
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ea3fa04b1cac0001b63d0d0'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

### Define a function to categorize the retrieved data


In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Create pandas dataframe from json output

In [18]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,Corrosion Service Company Limited,Construction & Landscaping,43.752432,-79.334661


In [19]:
nearby_venues.shape

(3, 4)

### Define a function to get the coordinates nearby venues

In [20]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Make a datafrmae with the venues

In [21]:
north_york_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park , Harbourfront
Lawrence Manor , Lawrence Heights
Queen's Park , Ontario Provincial Government
Islington Avenue
Malvern , Rouge
Don Mills
Parkview Hill , Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale
Rouge Hill , Port Union , Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood
Guildwood , Morningside , West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor , Wilson Heights , Downsview North
Thorncliffe Park
Richmond , Adelaide , King
Dufferin , Dovercourt Village
Scarborough Village
Fairview , Henry Farm , Oriole
Northwood Park , York University
East Toronto
Harbourfront East , Union Station , Toronto Islands
Little Portugal , Trinity
Kennedy Park , Ionview , East Birchmount Park
Bayview Village
Do

In [22]:
north_york_venues.shape

(2128, 7)

In [23]:
north_york_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


### Create a dataframe with number of venues in each neighborhood

In [24]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood , Long Branch",10,10,10,10,10,10
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",25,25,25,25,25,25
Berczy Park,57,57,57,57,57,57
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",22,22,22,22,22,22
Business reply mail Processing CentrE,18,18,18,18,18,18


In [25]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 268 uniques categories.


### Make numeric variable with each venue

In [26]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
north_york_onehot.shape

(2128, 268)

In [28]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
1,"Alderwood , Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
2,"Bathurst Manor , Wilson Heights , Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.050000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
3,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
4,"Bedford Park , Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
5,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017544,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
6,"Birch Cliff , Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
7,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
8,Business reply mail Processing CentrE,0.055556,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000
9,"CN Tower , King and Spadina , Railway Lands , ...",0.000000,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,...,0.00,0.000000,0.000000,0.000000,0.00000,0.00,0.000000,0.0000,0.0,0.000000


In [29]:
north_york_grouped.shape

(95, 268)

### Get frequency of top 5 venues in each neighborhood

In [30]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1               Skating Rink   0.2
2             Breakfast Spot   0.2
3  Latin American Restaurant   0.2
4             Clothing Store   0.2


----Alderwood , Long Branch----
                venue  freq
0         Pizza Place   0.2
1  Athletics & Sports   0.1
2        Dance Studio   0.1
3            Pharmacy   0.1
4         Coffee Shop   0.1


----Bathurst Manor , Wilson Heights , Downsview North----
                 venue  freq
0          Coffee Shop  0.10
1                 Bank  0.10
2           Restaurant  0.05
3  Fried Chicken Joint  0.05
4   Frozen Yogurt Shop  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1   Chinese Restaurant  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4  Monument / Landmark  0.00


----Bedford Park , Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.08
1      Sandwich Place  0.08
2  Ital

               venue  freq
0       Food Service   0.5
1     Baseball Field   0.5
2        Yoga Studio   0.0
3      Metro Station   0.0
4  Mobile Phone Shop   0.0


----Humewood-Cedarvale----
                             venue  freq
0                            Field  0.33
1                            Trail  0.33
2                     Hockey Arena  0.33
3  Molecular Gastronomy Restaurant  0.00
4                Mobile Phone Shop  0.00


----India Bazaar , The Beaches West----
                  venue  freq
0  Fast Food Restaurant  0.11
1               Brewery  0.06
2     Fish & Chips Shop  0.06
3    Italian Restaurant  0.06
4            Restaurant  0.06


----Kennedy Park , Ionview , East Birchmount Park----
              venue  freq
0  Department Store  0.17
1        Hobby Shop  0.17
2     Train Station  0.17
3       Coffee Shop  0.17
4    Discount Store  0.17


----Kensington Market , Chinatown , Grange Park----
                   venue  freq
0                   Café  0.09
1            

                           venue  freq
0                           Café  0.13
1                 Sandwich Place  0.13
2                    Coffee Shop  0.09
3                           Park  0.04
4  Vegetarian / Vegan Restaurant  0.04


----The Beaches----
               venue  freq
0  Health Food Store  0.25
1              Trail  0.25
2                Pub  0.25
3        Yoga Studio  0.00
4      Metro Station  0.00


----The Danforth West , Riverdale----
                    venue  freq
0        Greek Restaurant  0.19
1             Coffee Shop  0.07
2      Italian Restaurant  0.07
3          Ice Cream Shop  0.05
4  Furniture / Home Store  0.05


----The Kingsway , Montgomery Road , Old Mill North----
                        venue  freq
0                       River   0.5
1                        Park   0.5
2                 Men's Store   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create a dataframe with top ten venues in each neighborhood

In [33]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Lounge,Breakfast Spot,Clothing Store,Women's Store,Doner Restaurant,Diner,Discount Store,Distribution Center
1,"Alderwood , Long Branch",Pizza Place,Dance Studio,Gym,Coffee Shop,Skating Rink,Pharmacy,Pub,Athletics & Sports,Sandwich Place,Diner
2,"Bathurst Manor , Wilson Heights , Downsview North",Bank,Coffee Shop,Supermarket,Sushi Restaurant,Ice Cream Shop,Deli / Bodega,Middle Eastern Restaurant,Restaurant,Diner,Pizza Place
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Thai Restaurant,Sandwich Place,Italian Restaurant,Restaurant,Pizza Place,Butcher,Pub,Café,Pharmacy


In [34]:
neighborhoods_venues_sorted.shape

(95, 11)

## Clustering the North York's neighborhood

In [35]:
# set number of clusters
kclusters = 5

north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [38]:
north_york_data = df2[df2['Borough'] == 'North York'].reset_index(drop=True)
north_york_data.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


### Label the clusters and merged venues with coordinate data

In [42]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_york_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Construction & Landscaping,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Portuguese Restaurant,French Restaurant,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Women's Store,Furniture / Home Store,Accessories Store,Event Space,Gift Shop,Vietnamese Restaurant,Boutique,Coffee Shop,Miscellaneous Shop
3,M3B,North York,Don Mills,43.745906,-79.352188,1.0,Sporting Goods Shop,Asian Restaurant,Japanese Restaurant,Beer Store,Gym,Restaurant,Coffee Shop,Baseball Field,Dim Sum Restaurant,Supermarket
4,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Park,Pizza Place,Pub,Japanese Restaurant,Drugstore,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dance Studio


### Create folium map with the neighborhood clustersthe cluster 
### Check the cluster labels before plotting

In [51]:
# create map
north_york_merged['Cluster Labels']=north_york_merged['Cluster Labels'].fillna(1)
north_york_merged['Cluster Labels']=north_york_merged['Cluster Labels'].astype(int)
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighborhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Check the clusters
### Though we defined 5 cluster, we got only two clusters

In [53]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 1, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1,Portuguese Restaurant,French Restaurant,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Women's Store,Dessert Shop,Dim Sum Restaurant,Diner
2,North York,1,Clothing Store,Women's Store,Furniture / Home Store,Accessories Store,Event Space,Gift Shop,Vietnamese Restaurant,Boutique,Coffee Shop,Miscellaneous Shop
3,North York,1,Sporting Goods Shop,Asian Restaurant,Japanese Restaurant,Beer Store,Gym,Restaurant,Coffee Shop,Baseball Field,Dim Sum Restaurant,Supermarket
4,North York,1,Park,Pizza Place,Pub,Japanese Restaurant,Drugstore,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Dance Studio
5,North York,1,Sporting Goods Shop,Asian Restaurant,Japanese Restaurant,Beer Store,Gym,Restaurant,Coffee Shop,Baseball Field,Dim Sum Restaurant,Supermarket
6,North York,1,Golf Course,Dog Run,Mediterranean Restaurant,Pool,Dumpling Restaurant,Drugstore,Donut Shop,Doner Restaurant,Deli / Bodega,Eastern European Restaurant
7,North York,1,Bank,Coffee Shop,Supermarket,Sushi Restaurant,Ice Cream Shop,Deli / Bodega,Middle Eastern Restaurant,Restaurant,Diner,Pizza Place
8,North York,1,Clothing Store,Coffee Shop,Fast Food Restaurant,Bakery,Food Court,Japanese Restaurant,Cosmetics Shop,Bus Station,Restaurant,Bank
9,North York,1,Miscellaneous Shop,Caribbean Restaurant,Coffee Shop,Furniture / Home Store,Massage Studio,Bar,Distribution Center,Dim Sum Restaurant,Diner,Discount Store
10,North York,1,Café,Bank,Chinese Restaurant,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop


In [54]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 3, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Park,Food & Drink Shop,Construction & Landscaping,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
22,North York,3,Park,Bank,Convenience Store,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Drugstore
